In [1]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using Revise, Distributions, GLMakie, CSV, Dates
using TrypColonies

  Activating project at `c:\Users\Andreas\Dropbox\phd\TrypColonies\TrypColonies`


In [9]:
haskey(settings, "iterations23")

false

In [10]:
fields = String.(fieldnames(parameters))

("size", "agent_number", "interaction_radius", "velocity", "noise_dis", "noise_strength", "timesteps", "boundary_conditions", "geometry", "velocity_dis", "velocity_variance", "start_config", "radius_tanget", "radius_collision", "boundary_moveable", "path_tracing", "sliding_boundary", "grid_strength", "grid_recover_rate")

In [13]:
keys(settings)  

KeySet for a Dict{String, Any} with 22 entries. Keys:
  "radius_tanget"
  "boundary_conditions"
  "agent_number"
  "noise_dis"
  "sweep_parameter"
  "geometry"
  "parameter_increment"
  "noise_strength"
  "velocity_variance"
  "iterations"
  "parameter_steps"
  "velocity_dis"
  "size"
  "velocity"
  "radius_collision"
  "interaction_radius"
  "grid_strength"
  "start_config"
  "path_tracing"
  ⋮

In [21]:
function remove_fields_dict(dict)
    fields = String.(fieldnames(parameters))
    keys2 = keys(dict)  
    for key2 in keys2
        if key2 ∉ fields 
            pop!(dict, key2)
        end
    end
    return dict
end

remove_fields_dict (generic function with 1 method)

In [22]:
settings = Dict{String,Any}(
    "iterations"            => 2,
    "parameter_steps"       => 20,
    "sweep_parameter"       => "noise_strength",
    "parameter_increment"   => 0.1,
    "size"                  => (500,500),
    "agent_number"          => 1000,
    "timesteps"             => 1000,
    "interaction_radius"    => 7,
    "velocity"              => 5,
    "noise_dis"             => Normal,
    "noise_strength"        => 0.0,
    "boundary_conditions"   => "periodic",
    "geometry"              => "circle_small",
    "velocity_dis"          => Normal,
    "velocity_variance"     => 3.0,
    "start_config"          => "random", 
    "radius_tanget"         => 15,
    "radius_collision"      => 5,
    "boundary_moveable"     => true, 
    "path_tracing"          => false, 
    "sliding_boundary"      => false,
    "grid_strength"         => 20, 
    )


Dict{String, Any} with 22 entries:
  "radius_tanget"       => 15
  "boundary_conditions" => "periodic"
  "agent_number"        => 1000
  "noise_dis"           => Normal
  "sweep_parameter"     => "noise_strength"
  "geometry"            => "circle_small"
  "parameter_increment" => 0.1
  "noise_strength"      => 0.0
  "velocity_variance"   => 3.0
  "iterations"          => 2
  "parameter_steps"     => 20
  "velocity_dis"        => Normal
  "size"                => (500, 500)
  "velocity"            => 5
  "radius_collision"    => 5
  "interaction_radius"  => 7
  "grid_strength"       => 20
  "start_config"        => "random"
  "path_tracing"        => false
  ⋮                     => ⋮

In [23]:
remove_fields_dict(settings)    

Dict{String, Any} with 18 entries:
  "radius_tanget"       => 15
  "boundary_conditions" => "periodic"
  "agent_number"        => 1000
  "noise_dis"           => Normal
  "geometry"            => "circle_small"
  "noise_strength"      => 0.0
  "velocity_variance"   => 3.0
  "velocity_dis"        => Normal
  "size"                => (500, 500)
  "velocity"            => 5
  "radius_collision"    => 5
  "interaction_radius"  => 7
  "grid_strength"       => 20
  "start_config"        => "random"
  "path_tracing"        => false
  "timesteps"           => 1000
  "boundary_moveable"   => true
  "sliding_boundary"    => false

In [1]:
using SparseArrays, Serialization
using BenchmarkTools, LinearAlgebra
using ColorSchemes, GLMakie, DataStructures, Distributions

In [2]:
using Parameters

In [85]:
@benchmark replace(u -> (u>0) ? 0 : u, grid)

BenchmarkTools.Trial: 2104 samples with 1 evaluation.
 Range (min … max):  1.480 ms … 225.186 ms  ┊ GC (min … max):  0.00% … 99.05%
 Time  (median):     1.954 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.370 ms ±   4.963 ms  ┊ GC (mean ± σ):  15.99% ± 17.14%

  ▃▂    ▆█▆▄▂                                            ▃▂    
  ███▇▅▆██████▇▇█▇▆▆▅▄▃▄▁▃▃▆▃▃▄▁▁▁▁▁▃▁▁▁▅▃▁▁▁▁▁▃▁▁▁▅▃▄▅▁▇████ █
  1.48 ms      Histogram: log(frequency) by time      5.13 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 3.

In [131]:
grid = rand(-3:100,1000,1000)

1000×1000 Matrix{Int64}:
 28   81  92   2  13   26  -2  67  96  …   70  79  15  12  42  19  16  83  39
 86  100  52  37  81   23  96  54  37      45  80  38  48  35  43   2  98   8
 22   20  50  90  31   84  73  67  91     100  87  23  66  77  68  57  29  21
 84    0  25  53  15   83  10  63  82      29  50  88  13  26  92  92  93  68
 53   75  14  83  39  100   6  12  99      18  71  95   5  67  50  21  76   6
  6   14  25  91  13   44  16  55  20  …    7   4  12  99  11  30  47   1  63
 29   94  55  51  90   23   6  68  37      41  62  10  24  27  78  96  35  49
 41    8  94  90  86   23  74  88  84      14  35  15  57  50  26  40  26  83
 37    7  19  -2   3   28  51  35  15      83   1  75  23  84  14  54  52  18
 67   73  93  74  12    6  73  55  72      17  31  72  65  19  54  51  40  81
  ⋮                     ⋮              ⋱                    ⋮              
 97   51  91  99  55   92  42  19  25      54  33  18  73  37  63  84  89  20
 38   89  85  84  94   88  85   3  46    

In [4]:
hist(rand(Poisson(3.2), 10000))

In [12]:
hans = rand(Poisson(0.1), 10000)
unique(hans)    

5-element Vector{Int64}:
 0
 1
 2
 3
 4

In [13]:
a = hist(hans, bins=12, strokecolor = :black,strokewidth = 1, normalization = :pdf)
display(a)


GLMakie.Screen(...)

In [132]:
grid2 = replace(u -> (u>0) ? 0 : u, grid)

1000×1000 Matrix{Int64}:
 0   0  0   0   0  0  -2   0   0   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0   0  0   0   0  0   0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0   0  0   0   0  0   0   0   0  -3     0  0  0  0  0  0  0  0  0  0  0  0
 0   0  0   0   0  0   0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0   0  0   0   0  0   0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0   0  0   0   0  0   0   0   0   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0   0  0   0   0  0   0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0   0  0   0   0  0   0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0   0  0  -2   0  0   0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0   0  0   0   0  0   0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 ⋮                 ⋮                  ⋱        ⋮              ⋮           
 0   0  0   0   0  0   0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0   0  0   0   0  0   0   0   0   0     0  0  0  0  0  0  0  0 

In [129]:
grid_vec_sparse = Vector{SparseMatrixCSC{Int64, Int64}}(undef,0)

SparseMatrixCSC{Int64, Int64}[]

In [133]:
push!(grid_vec_sparse, sparse(grid2))

2-element Vector{SparseMatrixCSC{Int64, Int64}}:
 sparse([129, 132, 164, 182, 205, 206, 208, 254, 292, 300  …  563, 635, 663, 745, 755, 791, 884, 888, 904, 990], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000], [-1, -1, -1, -3, -1, -2, -1, -3, -1, -3  …  -3, -1, -1, -3, -3, -3, -1, -3, -1, -3], 1000, 1000)
 sparse([136, 143, 150, 191, 243, 330, 335, 354, 393, 403  …  676, 742, 755, 787, 796, 865, 877, 888, 907, 991], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000], [-1, -2, -3, -2, -1, -3, -3, -1, -3, -3  …  -3, -1, -2, -1, -2, -1, -2, -1, -2, -1], 1000, 1000)

In [90]:
grid_vec_sparse

2-element Vector{SparseMatrixCSC{Int64, Int64}}:
 sparse([1, 3, 5, 8, 9, 10, 14, 17, 18, 20  …  979, 980, 982, 983, 984, 989, 990, 994, 996, 999], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000], [-1, -3, -1, -1, -9, -3, -9, -9, -10, -4  …  -2, -10, -3, -1, -10, -3, -2, -5, -5, -10], 1000, 1000)
 sparse([3, 7, 8, 9, 10, 11, 13, 14, 15, 17  …  988, 989, 991, 992, 993, 995, 996, 997, 999, 1000], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000], [-1, -1, -1, -2, -8, -7, -1, -2, -4, -9  …  -5, -8, -1, -3, -9, -2, -2, -7, -1, -6], 1000, 1000)

In [135]:
a2 = rand(-1:2,1000,1000);

In [136]:
@benchmark a = Matrix(grid_vec_sparse[1])+grid_vec_sparse[1]

BenchmarkTools.Trial: 742 samples with 1 evaluation.
 Range (min … max):  4.261 ms … 235.903 ms  ┊ GC (min … max):  0.00% … 97.72%
 Time  (median):     5.694 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   6.730 ms ±   8.587 ms  ┊ GC (mean ± σ):  16.23% ± 16.00%

    ▁▁      ▄██▆▃                        ▅▅▄                   
  ▇████▅▆▆▄▆██████▆▆▁▄▅▁▁▁▁▄▁▁▁▅▄▁▅▁▆▅▅▁█████▇▁▅▁▄▁▁▁▁▁▄▄▁▁▄▄ █
  4.26 ms      Histogram: log(frequency) by time      10.9 ms <

 Memory estimate: 22.89 MiB, allocs estimate: 8.

In [137]:
a2

1000×1000 Matrix{Int64}:
  0   1   1   1  -1   0   2   2   1  …  -1   2  -1  -1   2   0   1   0   2
  1  -1  -1   2   0   1   1   2  -1      0   1   2   1   0   0   0   0   1
  1   0   0   0   2  -1   0  -1  -1      2   1   1   1   0   0   0   0  -1
  1   2   0   1   2  -1  -1   2   0      2   0  -1  -1   1  -1   0   1   2
  0   2  -1   1   0   0   0   1   2     -1   1  -1   1  -1   2   0   2   0
  1  -1   0   0  -1  -1   0   0   0  …   1   2   1  -1   2  -1  -1   1   2
  0  -1   1   1   0   0   2   1   1      2   2   2  -1   2   0   1   0   1
 -1   2   0   0   2  -1  -1  -1   1     -1   2   2   1  -1   0   2  -1   0
 -1   0   2   0   2   2  -1   2   0      1  -1  -1   0   2   0   1  -1  -1
 -1   2  -1   1   2   1  -1   0   1      2   0   0  -1   2   1   2   1   2
  ⋮                   ⋮              ⋱                   ⋮              
  0   0  -1   0  -1   1  -1   0   2      0   1   1   0  -1   2   0  -1  -1
  0  -1   1   2   2  -1  -1   2   2     -1   2  -1   1   1   1  -1  -1   0
 -

In [138]:
@benchmark begin 
    $a2 += grid_vec_sparse[1]   
end

BenchmarkTools.Trial: 1027 samples with 1 evaluation.
 Range (min … max):  3.228 ms … 239.140 ms  ┊ GC (min … max):  0.00% … 98.23%
 Time  (median):     4.144 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   4.862 ms ±   7.441 ms  ┊ GC (mean ± σ):  15.38% ± 16.06%

  ▂▁▂       ▇██▆▃                                        ▄▄▂▁  
  ████▁▄▄▅▅▇█████▇▆▇▅▅▅▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▄▁▁▁▄▅▄▁▁▁▇████ █
  3.23 ms      Histogram: log(frequency) by time      7.69 ms <

 Memory estimate: 15.26 MiB, allocs estimate: 5.

In [15]:
    C = findall(grid.>0)
    grid[C] .= 0
grid == grid2

true

In [195]:
function save_grid_changes!(Grid_1::Union{Matrix{Int64}, Real},Grid_2::Matrix{Int64}, Grid_vec::Vector{SparseMatrixCSC{Int64, Int64}})
    push!(Grid_vec,Grid_2 .- Grid_1)
end

save_grid_changes! (generic function with 2 methods)

In [196]:
ones(10,10) .-0

10×10 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [197]:
y_size, x_size = 1000, 1000    

(1000, 1000)

In [208]:

start_grid  

1000×1000 Matrix{Int64}:
 0  0   0   0   0   0   0  0  0  0  …   0  -1   0   0   0  0  0   0   0  0
 0  0   0   0   0   0   0  0  0  0      0   0   0   0   0  0  0   0   0  0
 0  0   0   0   0  -2   0  0  0  0      0   0   0   0   0  0  0   0   0  0
 0  0   0   0   0   0   0  0  0  0      0   0   0   0   0  0  0   0   0  0
 0  0   0   0   0   0   0  0  0  0      0   0   0   0   0  0  0   0   0  0
 0  0   0   0   0   0   0  0  0  0  …   0   0   0   0   0  0  0   0   0  0
 0  0   0   0   0   0  -1  0  0  0      0   0   0   0  -2  0  0   0   0  0
 0  0   0   0   0   0   0  0  0  0      0   0   0   0   0  0  0   0   0  0
 0  0   0   0   0   0   0  0  0  0      0   0   0   0   0  0  0   0   0  0
 0  0   0   0   0   0   0  0  0  0      0   0   0   0   0  0  0   0   0  0
 ⋮                  ⋮               ⋱   ⋮                  ⋮             
 0  0   0  -1   0   0   0  0  0  0      0   0   0   0   0  0  0   0   0  0
 0  0   0   0   0   0   0  0  0  0      0   0   0   0   0  0  0   0  -2  0
 

In [209]:
grid_vec = Vector{Matrix{Int64}}(undef,0)
grid_vec_sparse =Vector{SparseMatrixCSC{Int64, Int64}}(undef,0)
start_grid_frozen = zeros(Int64,y_size,x_size)
    start_grid = rand(-3:100,y_size,x_size)
    replace!( u -> (u>0) ? 0 : u, start_grid)
    save_grid_changes!(0,start_grid,grid_vec_sparse)
    for i in 1:999
        grid_1 = deepcopy(start_grid)
        for changes in 1:100
            start_grid[rand(1:y_size),rand(1:x_size)] += -1
        end
        #push!(grid_vec, deepcopy(start_grid))
        save_grid_changes!(grid_1,start_grid,grid_vec_sparse)
    end


In [211]:
grid_vec_sparse;

In [213]:
function reconstruct_grid(start_grid::Matrix{Int64},grid_vec_sparse::Vector{SparseMatrixCSC{Int64, Int64}}, timestep::Int64)
    gridi = deepcopy(start_grid)
    for i in 1:timestep
        gridi .+= grid_vec_sparse[i]
    end
    return gridi
end

reconstruct_grid (generic function with 1 method)

In [214]:
function reconstruct_grid_2(start_grid::Matrix{Int64},grid_vec_sparse::Vector{SparseMatrixCSC{Int64, Int64}}, timestep::Int64)
    gridi = deepcopy(start_grid)

    for t in 1:timestep
        non_zeros_index = findnz(grid_vec_sparse[t])
        for i in 1:length(non_zeros_index[1])
             gridi[non_zeros_index[1][i],non_zeros_index[2][i]] += non_zeros_index[3][i]
        end
    end
    return gridi
end

reconstruct_grid_2 (generic function with 1 method)

In [215]:
non_zeros_index = findnz(grid_vec_sparse[1])

([19, 84, 126, 138, 182, 270, 277, 287, 346, 364  …  861, 866, 870, 877, 885, 888, 911, 951, 954, 992], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000], [-1, -2, -1, -2, -1, -1, -3, -3, -1, -3  …  -1, -2, -1, -3, -2, -2, -1, -3, -1, -3])

In [216]:
function update_grid!(Grid,Grid_vec_sparse,Timestep;start = Timestep)
    non_zeros_index = findnz(Grid_vec_sparse[Timestep])
    for t in start:Timestep
        non_zeros_index = findnz(Grid_vec_sparse[t])
        for i in 1:length(non_zeros_index[1])
            Grid[non_zeros_index[1][i],non_zeros_index[2][i]] += non_zeros_index[3][i]
        end
    end
    return Grid
end


update_grid! (generic function with 1 method)

In [235]:
function construct_grid(Grid_vec_sparse::Vector{SparseMatrixCSC{Int64, Int64}},Timestep::Int; start_grid::AbstractArray{Int} = Matrix(Grid_vec_sparse[1]))
    for t in 2:Timestep
        non_zeros_index = findnz(Grid_vec_sparse[t])
        for i in 1:length(non_zeros_index[1])
            start_grid[non_zeros_index[1][i],non_zeros_index[2][i]] += non_zeros_index[3][i]
        end
    end
    return start_grid
end

construct_grid (generic function with 3 methods)

In [231]:
start_grid_frozen_fr = deepcopy(start_grid_frozen)

reconstruct_grid_2(start_grid_frozen, grid_vec_sparse,100) ==update_grid!(start_grid_frozen_fr,grid_vec_sparse,100,start = 1)

true

In [237]:
@benchmark construct_grid(grid_vec_sparse,999, start_grid = hans)

BenchmarkTools.Trial: 1366 samples with 1 evaluation.
 Range (min … max):  2.485 ms … 10.438 ms  ┊ GC (min … max): 0.00% … 59.17%
 Time  (median):     3.550 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.652 ms ±  1.217 ms  ┊ GC (mean ± σ):  5.08% ± 10.62%

    ▁▃▅▂    █▇▂                                               
  ▅█████▆▅▅████▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃▂▃ ▃
  2.49 ms        Histogram: frequency by time        9.99 ms <

 Memory estimate: 2.56 MiB, allocs estimate: 2996.

In [218]:
@benchmark update_grid!(start_grid_frozen_fr,grid_vec_sparse,999,start = 1)

BenchmarkTools.Trial: 1269 samples with 1 evaluation.
 Range (min … max):  2.835 ms … 55.133 ms  ┊ GC (min … max): 0.00% … 92.82%
 Time  (median):     3.614 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.929 ms ±  1.673 ms  ┊ GC (mean ± σ):  4.10% ±  8.93%

      ▁▃█▄▁      ▂▄▃▁                                         
  ▃▄▆██████▆▅▃▃▄▅████▆▄▃▃▂▁▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃▃▂▂▂▂ ▃
  2.84 ms        Histogram: frequency by time         7.8 ms <

 Memory estimate: 3.22 MiB, allocs estimate: 3003.

In [219]:
i = 999
@benchmark reconstruct_grid(start_grid_frozen, grid_vec_sparse,i)

BenchmarkTools.Trial: 6 samples with 1 evaluation.
 Range (min … max):  892.330 ms … 910.577 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     893.537 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   898.417 ms ±   8.326 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █ ███                                             █         █  
  █▁███▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█ ▁
  892 ms           Histogram: frequency by time          911 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 4.

In [220]:
i = 999
@benchmark reconstruct_grid_2(start_grid_frozen, grid_vec_sparse,i)

BenchmarkTools.Trial: 830 samples with 1 evaluation.
 Range (min … max):  5.218 ms … 20.551 ms  ┊ GC (min … max): 0.00% … 38.05%
 Time  (median):     5.704 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.016 ms ±  1.038 ms  ┊ GC (mean ± σ):  4.93% ±  9.90%

  ▁▂▃▁  ▂▆█▆▄▃                                          ▁▃▁   
  ████▇▇███████▅▄▅▅▁▁▅▁▁▁▄▄▁▁▁▄▅▅▅▁▁▁▄▁▄▄▁▄▁▁▁▁▄▄▁▇▅▅▅▁▇███▇ █
  5.22 ms      Histogram: log(frequency) by time     8.44 ms <

 Memory estimate: 10.85 MiB, allocs estimate: 3004.

In [170]:
hans = sparse(zeros(Int64,500,500))
@benchmark begin
    for i in 1:50
        hans[rand(1:500),rand(1:500)] = rand(1:100)
    end
end

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):    9.200 μs …   2.207 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     261.850 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   304.383 μs ± 229.295 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                              
  █▆▇▇█▆▆▇▇▇██▇▇▆▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁ ▃
  9.2 μs           Histogram: frequency by time          985 μs <

 Memory estimate: 2.34 KiB, allocs estimate: 50.

In [172]:
hans2 = zeros(Int64,500,500)
@benchmark begin
    for i in 1:50
        hans2[rand(1:500),rand(1:500)] = rand(1:100)
    end
end

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.378 μs … 14.033 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.411 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.982 μs ±  1.512 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▂         ▄                                           ▄   ▁
  ██▆▅▅▄▁▄▅▅▆██▇▄▃▅▄▁▁▄▆▄▅▄▁▁▁▁▁▃▁▄▃▄▄▃▃▄▁▃▃▃▁▃▁▄▃▁▁▁▁▁▁██▇▅ █
  2.38 μs      Histogram: log(frequency) by time     7.51 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [174]:
@benchmark deepcopy(hans2)

BenchmarkTools.Trial: 9540 samples with 1 evaluation.
 Range (min … max):  316.100 μs … 244.349 ms  ┊ GC (min … max):  0.00% … 99.80%
 Time  (median):     416.000 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   521.179 μs ±   2.551 ms  ┊ GC (mean ± σ):  18.64% ± 12.44%

  ▄█▂            ▁                                              ▁
  ███████▄▆▅▄▄▄▃▁█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▅██ █
  316 μs        Histogram: log(frequency) by time       3.94 ms <

 Memory estimate: 1.91 MiB, allocs estimate: 4.

test to construct 2d vectors out of 2 lists

In [44]:
x_lot = rand(1:10,10)
y_lot = rand(-10:-3,10)

vec_array = [[x_lot[i],y_lot[i]] for i in 1:length(x_lot)]

10-element Vector{Vector{Int64}}:
 [1, -6]
 [6, -8]
 [3, -10]
 [4, -5]
 [9, -6]
 [1, -3]
 [4, -10]
 [1, -8]
 [8, -6]
 [10, -4]

In [45]:
rot_mat =  [0 -1; 1 0]
rot_mat_list = fill(rot_mat, length(y_lot))
new_dir = rot_mat_list.*vec_array

10-element Vector{Vector{Int64}}:
 [6, 1]
 [8, 6]
 [10, 3]
 [5, 4]
 [6, 9]
 [3, 1]
 [10, 4]
 [8, 1]
 [6, 8]
 [4, 10]

In [46]:
new_dir_norm = normalize.(new_dir)

10-element Vector{Vector{Float64}}:
 [0.9863939238321437, 0.1643989873053573]
 [0.8, 0.6000000000000001]
 [0.9578262852211514, 0.2873478855663454]
 [0.7808688094430304, 0.6246950475544243]
 [0.5547001962252291, 0.8320502943378436]
 [0.9486832980505138, 0.31622776601683794]
 [0.9284766908852593, 0.3713906763541037]
 [0.9922778767136677, 0.12403473458920847]
 [0.6000000000000001, 0.8]
 [0.3713906763541037, 0.9284766908852593]

In [47]:
dot.(new_dir,vec_array)

10-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [48]:
dot.(new_dir_norm,new_dir_norm)

10-element Vector{Float64}:
 1.0
 1.0000000000000002
 1.0
 1.0
 0.9999999999999999
 0.9999999999999999
 1.0
 1.0000000000000002
 1.0000000000000002
 1.0

In [49]:
η = π/2

rot_mat = [cos(η) -sin(η); sin(η) cos(η)]

2×2 Matrix{Float64}:
 6.12323e-17  -1.0
 1.0           6.12323e-17

In [50]:
rot_mat_90 =  [0 -1; 1 0]

2×2 Matrix{Int64}:
 0  -1
 1   0

In [51]:
using GLMakie

In [52]:
vec1 = [-1.0,5.0]


2-element Vector{Float64}:
 -1.0
  5.0

In [53]:
vec1 = rot_mat_90*vec1

2-element Vector{Float64}:
 -5.0
 -1.0

In [54]:
arrows([0.0],[0.0],[vec1[1]],[vec1[2]])

In [55]:
typeof(theme_ggplot2())

Attributes

In [56]:
struct function_list
    function1::Function
    function2::Function
end

In [57]:
fu_l = function_list(push!,append!)

function_list(push!, append!)

In [58]:
function_dic = Dict("z"=>4, push! =>"push!")

Dict{Any, Any} with 2 entries:
  push! => "push!"
  "z"   => 4

In [59]:
fu_l.function1

push! (generic function with 93 methods)

In [60]:
function_dic[fu_l.function1]

"push!"

In [61]:
String("push!()")

"push!()"

In [62]:
fu_l.function1([1,1],4)

3-element Vector{Int64}:
 1
 1
 4

In [63]:
function test_return(;a = 2)
    if a == 2
        return 1,2
    else
        return 1,2,3
    end
end

test_return (generic function with 1 method)

In [64]:
a,b = test_return(a =3)

(1, 2, 3)

In [65]:
a


1

In [66]:
b

2

In [67]:
for i in 1:3
    println(i)
end

1
2
3
